# Caribbean reforestation potential

This is code for Southeast Conservation Blueprint 2023. Written by Rua Mordecai.

## Import libraries and define variables

In [75]:
import os
import arcpy

In [76]:
# define spatial reference, workspaces, and source data location
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\CaribbeanBlueprint2023_FinalIndicators\CaribbeanBlueprint2023_FinalIndicators\SpatialData"
landfire = r"D:\Landcover\LF2020_Puerto_Rico_Virgin_Islands_220_IA\LF2020_PRVI_220_IA\LF2020_EVT_220_PRVI\Tif\LV20_EVT_220.tif"
wbd = r"C:\GIS\boundaries\WBD_National_GDB\WBD_National_GDB.gdb"

In [77]:
# define final output name
IndicatorFileName = "CaribbeanReforestationPotential.tif"

In [78]:
# define raster used for cell size, extent, and snapping
CaribbeanRaster= r"D:\Blueprint\2023\extent\VIPR_Extent_v6.tif"

In [79]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

## Start analysis

### Prep data

In [29]:
# Set the source workspace for watershed boundaries
arcpy.env.workspace = wbd

In [30]:
# Export HUC12s in Caribbean
arcpy.conversion.FeatureClassToFeatureClass("WBDHU12",OutputWorkspace,"CaribbeanHUC12.shp","states = 'VI' Or states = 'PR'")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Reforestation\\working\\CaribbeanHUC12.shp'>

In [80]:
# Switch to working directory
arcpy.env.workspace = OutputWorkspace

### Begin analysis

In [32]:
# Make raster to identify coffee (Caribbean bush fruit and berries,Tropical agroforestry plantation)
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Reclassify(landfire, "Value", "-9999 NODATA;7292 7755 0;7838 1;7861 9354 0;9355 1; 9803 0", "DATA"); out_raster.save("coffee.tif")

In [33]:
# Make raster to identify potential reforestation areas. Include Developed-Open Space, Agriculture, and 
# coffee classes
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Reclassify(landfire, "Value", "-9999 NODATA;7292 7300 0;7754 7838 1;7861 9354 0;9355 1; 9803 0", "DATA"); out_raster.save("potentialReforest.tif")

In [34]:
# Get HUC12s with Reservoirs by looking for Dam in the subwatershed name. Waterbody type doesn't work well here but this
# approach does.
out_raster = arcpy.conversion.FeatureClassToFeatureClass("CaribbeanHUC12.shp", OutputWorkspace, "ReservoirHUC12.shp", "name LIKE '%Dam%'")

In [35]:
# add a field to convert to raster, give a value of 100
arcpy.management.CalculateField("ReservoirHUC12.shp", "raster", "100", "PYTHON3", '', "SHORT")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Reforestation\\working\\ReservoirHUC12.shp'>

In [36]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    arcpy.conversion.FeatureToRaster(in_features="ReservoirHUC12.shp", field="raster", out_raster="ReservoirHUC12Null.tif", cell_size=CaribbeanRaster)

In [37]:
# change raster so there isn't noData on land
out_raster = arcpy.sa.Con(arcpy.sa.IsNull("ReservoirHUC12Null.tif"),0,"ReservoirHUC12Null.tif"); out_raster.save("ReservoirHUC12.tif")

In [38]:
# add rasters together
output_raster = arcpy.ia.RasterCalculator(["ReservoirHUC12.tif","potentialReforest.tif","coffee.tif"],['a','b','c'],'a + b + c'); output_raster.save("reforestSum.tif")

In [39]:
# Reclassify for indicator categories
out_raster = arcpy.sa.Reclassify("reforestSum.tif", "Value", "0 0;1 1;2 3;100 0;101 2;102 4", "DATA"); out_raster.save("reforestBins.tif")

In [40]:
# clip raster to Caribbean extent
out_raster = arcpy.sa.ExtractByMask("reforestBins.tif", CaribbeanRaster); out_raster.save(IndicatorFileName)

## Finalize indicator

In [81]:
# set code block for legend
codeblock = """
def Reclass(value):
    if value == 0:
        return '0 = Low reforestation potential (already natural or developed)'    
    elif value == 1:
        return '1 = Medium reforestation potential (other open space in developed area, pasture, or crops)'
    elif value == 2:
        return '2 = High reforestation potential (open space in developed area, pasture, or crops in watershed with reservoir)' 
    elif value == 3:
        return '3 = Very high reforestation potential (other coffee production)' 
    elif value == 4:
        return '4 = Highest reforestation potential (coffee production in watershed with reservoir)'        
"""

In [82]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\CaribbeanBlueprint2023_FinalIndicators\\CaribbeanBlueprint2023_FinalIndicators\\SpatialData\\CaribbeanReforestationPotential.tif'>

In [85]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 4:
		return 0
	if Value == 3:
		return 77
	if Value == 2:
		return 202
	if Value == 1:
		return 250
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 4:
		return 110
	if Value == 3:
		return 162
	if Value == 2:
		return 191
	if Value == 1:
		return 205
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 4:
		return 118
	if Value == 3:
		return 165
	if Value == 2:
		return 187
	if Value == 1:
		return 196
	if Value == 0:
		return 255
	else:
		return 255
		
"""

In [86]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\CaribbeanBlueprint2023_FinalIndicators\\CaribbeanBlueprint2023_FinalIndicators\\SpatialData\\CaribbeanReforestationPotential.tif'>